In [2]:
import uclchem
from uclchem.makerates.network import Network, LoadedNetwork
from uclchem.makerates.species import Species
from uclchem.makerates.reaction import Reaction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# load species
species = pd.read_csv("../src/uclchem/species.csv")
species = [Species(list(spec)) for idx, spec in species.iterrows()]

In [4]:
# load reactions
reactions = pd.read_csv("../src/uclchem/reactions.csv")
# fix annoying electron freezeout
reactions.iloc[527, 3] = ""
reactions = [Reaction(list(reac)) for idx, reac in reactions.iterrows()]

In [5]:
reactions[0]

BULKSWAP reaction: @C -> #C

In [6]:
network = LoadedNetwork(species, reactions)

In [7]:
# print("Running test models...")
# # set a parameter dictionary for static model
# outSpecies = ["OH", "OCS", "CO", "CS", "CH3OH"]
# params = {
#     "endAtFinalDensity": False,
#     "freefall": False,
#     "writeStep": 1,
#     "initialDens": 1e4,
#     "initialTemp": 10.0,
#     "finalDens": 1e5,
#     "finalTime": 5.0e6,
#     "outputFile": "examples/test-output/static-full.dat",
#     "abundSaveFile": "examples/test-output/startstatic.dat",
#     "rateFile": "static_rates.csv",
# }
# uclchem.model.cloud(param_dict=params, out_species=outSpecies, return_rates=True)

# # change to collapsing phase1 params
# params["freefall"] = True
# params["endAtFinalDensity"] = True
# params["initialDens"] = 1e2
# params["abundSaveFile"] = "examples/test-output/startcollapse.dat"
# params["outputFile"] = "examples/test-output/phase1-full.dat"
# params["columnFile"] = "examples/test-output/phase1-column.dat"
# params["rateFile"] = "stage1_rates.csv"
# uclchem.model.cloud(param_dict=params, out_species=outSpecies, return_rates=True)

# # finally, run phase 2 from the phase 1 model.
# params["initialDens"] = 1e5
# params["freezeFactor"] = 0.0
# params["thermdesorb"] = True
# params["endAtFinalDensity"] = False
# params["freefall"] = False
# params["finalTime"] = 1e6
# params.pop("abundSaveFile")
# params["abundLoadFile"] = "examples/test-output/startcollapse.dat"
# params["outputFile"] = "examples/test-output/phase2-full.dat"
# params.pop("columnFile")
# params["rateFile"] = "stage2_rates.csv"
# uclchem.model.hot_core(
#     3, 300.0, param_dict=params, out_species=outSpecies, return_rates=True
# )

In [8]:
print("Running test models...")
# set a parameter dictionary for static model
outSpecies = ["OH", "OCS", "CO", "CS", "CH3OH"]
params = {
    "endAtFinalDensity": False,
    "freefall": False,
    "writeStep": 1,
    "initialDens": 1e4,
    "initialTemp": 10.0,
    "finalDens": 1e5,
    "finalTime": 5.0e6,
}
static_phys, static_chem, static_rates, _, succesflag = uclchem.model.cloud(
    param_dict=params, out_species=outSpecies, return_dataframe=True, return_rates=True
)

# change to collapsing phase1 params
params["freefall"] = True
params["endAtFinalDensity"] = True
params["initialDens"] = 1e2
phase1_phys, phase1_chem, phase1_rates, start_abunds, succesflag = uclchem.model.cloud(
    param_dict=params, out_species=outSpecies, return_dataframe=True, return_rates=True
)

# finally, run phase 2 from the phase 1 model.
params["initialDens"] = 1e5
params["freezeFactor"] = 0.0
params["thermdesorb"] = True
params["endAtFinalDensity"] = False
params["freefall"] = False
params["finalTime"] = 1e6
params["rateFile"] = "stage2_rates.csv"
phase2_phys, phase2_chem, phase2_rates, _, succesflag = uclchem.model.hot_core(
    3,
    300.0,
    param_dict=params,
    out_species=outSpecies,
    return_dataframe=True,
    return_rates=True,
    starting_chemistry=start_abunds,
)

Running test models...
 At T(=R1) and step size H(=R2), the                                             
 corrector convergence failed repeatedly                                         
 or with ABS(H) = HMIN.                                                          
In the above message, R1 =   0.7286403283401D+13   R2 =   0.3699694850023D+03
 ISTATE -5 - shortening step at time   230582.38238611323      years


In [9]:
# TODO: fix the string that is attached to reactions.
phase2_rates

,@C + BULKSWAP -> #C,@C2 + BULKSWAP -> #C2,@C2H + BULKSWAP -> #C2H,@C2H2 + BULKSWAP -> #C2H2,@C2H3 + BULKSWAP -> #C2H3,@C2H4 + BULKSWAP -> #C2H4,@C2H5 + BULKSWAP -> #C2H5,@C2N + BULKSWAP -> #C2N,@C3H2 + BULKSWAP -> #C3H2,@C3N + BULKSWAP -> #C3N,...,SO+ + E- -> S + O,SO+ + C2H2 -> H2CS+ + CO,SO+ + C2H2 -> HCO+ + HCS,SO+ + C2H2 -> HCS+ + HCO,SO+ + C2H4 -> H2CS+ + H2CO,SO+ + C2H4 -> H2COH+ + HCS,SO+ + C2H4 -> H2CSH+ + HCO,SO+ + OCS -> S2+ + CO2,SO2+ + E- -> S + O + O,SO2+ + E- -> SO + O
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,2.142504e-231,1.514979e-231,1.484370e-231,1.455545e-231,1.428336e-231,1.402598e-231,1.378203e-231,1.203983e-231,1.203983e-231,1.049608e-231,...,1.095445e-06,6.600000e-10,1.100000e-11,2.420000e-10,2.200000e-10,6.600000e-11,6.600000e-10,2.690000e-10,1.049436e-06,1.647439e-06
2,2.142505e-231,1.514980e-231,1.484371e-231,1.455546e-231,1.428337e-231,1.402599e-231,1.378204e-231,1.203984e-231,1.203984e-231,1.049609e-231,...,1.095445e-06,6.600000e-10,1.100000e-11,2.420000e-10,2.200000e-10,6.600000e-11,6.600000e-10,2.690000e-10,1.049436e-06,1.647439e-06
3,2.142512e-231,1.514985e-231,1.484376e-231,1.455551e-231,1.428342e-231,1.402604e-231,1.378209e-231,1.203988e-231,1.203988e-231,1.049612e-231,...,1.095445e-06,6.600000e-10,1.100000e-11,2.420000e-10,2.200000e-10,6.600000e-11,6.600000e-10,2.690000e-10,1.049436e-06,1.647439e-06
4,2.142564e-231,1.515021e-231,1.484412e-231,1.455585e-231,1.428376e-231,1.402637e-231,1.378242e-231,1.204017e-231,1.204017e-231,1.049637e-231,...,1.095445e-06,6.600000e-10,1.100000e-11,2.420000e-10,2.200000e-10,6.600000e-11,6.600000e-10,2.690000e-10,1.049436e-06,1.647439e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000e-07,6.600000e-10,1.100000e-11,2.420000e-10,2.200000e-10,6.600000e-11,6.600000e-10,2.690000e-10,1.790000e-07,2.810000e-07
190,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000e-07,6.600000e-10,1.100000e-11,2.420000e-10,2.200000e-10,6.600000e-11,6.600000e-10,2.690000e-10,1.790000e-07,2.810000e-07
191,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000e-07,6.600000e-10,1.100000e-11,2.420000e-10,2.200000e-10,6.600000e-11,6.600000e-10,2.690000e-10,1.790000e-07,2.810000e-07
192,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000e-07,6.600000e-10,1.100000e-11,2.420000e-10,2.200000e-10,6.600000e-11,6.600000e-10,2.690000e-10,1.790000e-07,2.810000e-07


In [10]:
from uclchem.analysis import postprocess_rates_to_dy

In [11]:
dy, flux = postprocess_rates_to_dy(
    phase2_phys, phase2_chem, phase2_rates, network=network
)